In [30]:
import pandas as pd
import numpy as np
import math
import datetime as datetime
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('FixedLatLongTimeLoc.csv')

/home/march/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data['DATE_TIME'] =  pd.to_datetime(data['DATE_TIME'], format='%Y-%m-%d %H:%M:%S')

In [4]:
data.head()

,Unnamed: 0,CALL_NO,UNIT_DISPATCH_DATE,DISPATCH_TIME,CALL_TYPE_FINAL,CALL_TYPE_FINAL_D,XCOORD,YCOORD,DATE_TIME
0,0,61209515,2006-12-28,44548,773A,UNCONSCIOUS PERS,39.824332,-86.236119,2006-12-28 04:45:48
1,1,61209516,2006-12-28,44853,774A,ASSAULT/TRAUMA,39.781627,-86.120826,2006-12-28 04:48:53
2,2,61209517,2006-12-28,45219,768A,SEIZURE,39.756840,-85.998714,2006-12-28 04:52:19
3,3,61209519,2006-12-28,51057,773A,UNCONSCIOUS PERS,39.782064,-86.044592,2006-12-28 05:10:57
4,4,61209520,2006-12-28,51756,790A,*MEDICAL ALARM,39.742415,-86.111559,2006-12-28 05:17:56


In [5]:
def assign_to_grid(xcord, ycord, xgridsize, ygridsize, xmin, xrange, ymin, yrange):
    # Assign a lat and long coordinate to a grid location. Input xcord, ycord as lat/long to be mapped
    # gridsize is dimension of matrix
    # min is smallest lat/long coordinate, range is max(xcoordinate) - min(xcoordinate) and repeat for y
    
    xbin = int((xcord-xmin)/(xmax-xmin)*xgridsize)
    ybin = int((ycord-ymin)/(ymax-ymin)*ygridsize)
    return xbin, ybin

#points = []
#for i in range(0, 60000):
#    point = assign_to_grid(data['XCOORD'][i], data['YCOORD'][i], 10, 10, xmin, xrange, ymin, yrange)
#    points.append(point)
#plt.plot(*zip(*points), marker='o', color='b', ls='')


In [101]:
def event_update(mu_g, Lam_g, time_delta_last_g_time, theta, w, F_g):
# update mu, theta and triggering exponentials for grids where event occurred
    # question why are we fixing dt instead of calculating it?
    dt = 0.01
    mu_g = mu_g + dt * (mu_g/Lam_g-mu_g*time_delta_last_g_time)
    for i in range(0, len(w)):
        theta[i] = theta[i] + dt * (F_g[i]/Lam_g-theta[i])
        F_g[i] = F_g[i] + w[i] * theta[i]
    return mu_g, theta, F_g

In [87]:
def global_update(Fjk, w, time_delta_last_time, theta, mu_jk):
# Decay triggering exponentials and calculate current intensities for all grids
    for i in range(0, len(w)):
        # question the + theta[i]*w[i] wasn't in the matlab code?
        Fjk[i] = Fjk[i] * np.exp(-1*w[i]*time_delta_last_time) + theta[i]*w[i]
    Lam_jk = mu_jk*sum(Fjk)
    
    return Fjk, Lam_jk

In [102]:

xmax =  data['XCOORD'].max()
xmin =  data['XCOORD'].min()
xrange = xmax-xmin
ymax =  data['YCOORD'].max()
ymin =  data['YCOORD'].min()
yrange = ymax-ymin

xgridsize = 100
ygridsize = 100

w = [.5, .1, .05]
theta = [0, 0, 0]

mu = np.ones([xgridsize, ygridsize])
F = np.ones([xgridsize, ygridsize, len(w)])
Lam = np.ones([xgridsize, ygridsize])

G_times = np.zeros([xgridsize, ygridsize])

for i in range(1, 2):
    [gx, gy] = assign_to_grid(data.XCOORD[i], data.YCOORD[i], xgridsize, ygridsize, xmin, xrange, ymin, yrange)
    for j in range(xgridsize):
        for k in range(ygridsize):
            time_delta = (data.DATE_TIME[i]- data.DATE_TIME[i-1]).total_seconds()
            [F[j][k], Lam[j][k]] = global_update(F[j][k], w, time_delta, theta, mu[j][k])
            
    g_time_delta = (data.DATE_TIME[i] - pd.to_datetime(G_times[gx, gy])).total_seconds()
    [mu[gx][gy], theta, F[gx][gy]] = event_update(mu[gx][gy], Lam[gx][gy], g_time_delta, theta, w, F[gx][gy])

called


In [107]:
mu

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])